In [1]:

import customtkinter as ctk
import tkinter as tk
from tkinter import filedialog, messagebox
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import random
from datetime import datetime
import os
import threading

URL_EMAKTAB_LOGIN = "https://login.emaktab.uz/"
CHROME_DRIVER_PATH = "/home/eclipser/Downloads/chromedriver"

class ModernLoginAutomation:
    def __init__(self):
        # Setup the main window
        self.app = ctk.CTk()
        self.app.title("Login Automation Dashboard")
        self.app.geometry("1000x800")
        
        # Configure grid layout
        self.app.grid_columnconfigure(0, weight=1)
        self.app.grid_rowconfigure(0, weight=1)
        
        # Variables
        self.file_path = None
        self.is_running = False
        self.current_progress = 0
        self.sidebar_open = True
        
        # Create main container
        self.create_main_frame()
        self.create_sidebar()
        self.create_main_content()
        
        # Theme settings
        ctk.set_appearance_mode("dark")
        ctk.set_default_color_theme("blue")

    def create_main_frame(self):
        self.main_frame = ctk.CTkFrame(self.app)
        self.main_frame.grid(row=0, column=0, sticky="nsew", padx=0, pady=0)
        self.main_frame.grid_columnconfigure(1, weight=1)
        self.main_frame.grid_rowconfigure(0, weight=1)

    def create_sidebar(self):
        # Sidebar frame with matching dark theme color
        self.sidebar = ctk.CTkFrame(self.main_frame, width=250)
        self.sidebar.grid(row=0, column=0, sticky="nsew", padx=0)
        
        # Hamburger menu button with matching theme color
        self.hamburger_button = ctk.CTkButton(
            self.app, 
            text="☰", 
            command=self.toggle_sidebar, 
            width=30, 
            height=30,
            fg_color="transparent",
            hover_color=("gray70", "gray30")
        )
        self.hamburger_button.place(x=10, y=10)
        
        # App title
        self.logo_label = ctk.CTkLabel(
            self.sidebar, 
            text="Login Automation",
            font=ctk.CTkFont(size=20, weight="bold")
        )
        self.logo_label.grid(row=1, column=0, padx=20, pady=(20, 10))
        
        # Theme switch
        self.theme_switch = ctk.CTkSwitch(
            self.sidebar, 
            text="Qora Mavzu",
            command=self.toggle_theme
        )
        self.theme_switch.grid(row=2, column=0, padx=20, pady=10)
        self.theme_switch.select()
        
        # Navigation buttons
        self.nav_button_1 = ctk.CTkButton(
            self.sidebar, 
            text="Boshqaruv Paneli",  # "Dashboard" in Uzbek
            command=self.show_dashboard,
            width=180,
            height=40
        )
        self.nav_button_1.grid(row=3, column=0, padx=20, pady=10)
        
        # Version label
        self.version_label = ctk.CTkLabel(
            self.sidebar, 
            text="v1.0.0",
            font=ctk.CTkFont(size=12)
        )
        self.version_label.grid(row=4, column=0, padx=20, pady=(10, 20))

    def create_main_content(self):
        # Main content frame
        self.content = ctk.CTkFrame(self.main_frame)
        self.content.grid(row=0, column=1, sticky="nsew", padx=0)
        self.content.grid_columnconfigure(0, weight=1)
        self.content.grid_rowconfigure(3, weight=1)
        
        # File selection section
        self.file_frame = ctk.CTkFrame(self.content)
        self.file_frame.grid(row=0, column=0, sticky="ew", padx=20, pady=(20, 10))
        self.file_frame.grid_columnconfigure(0, weight=1)
        
        self.file_label = ctk.CTkLabel(
            self.file_frame, 
            text="Fayl tanlanmagan",  # "No file selected" in Uzbek
            font=ctk.CTkFont(size=14)
        )
        self.file_label.grid(row=0, column=0, sticky="w", padx=20, pady=10)
        
        self.browse_button = ctk.CTkButton(
            self.file_frame, 
            text="Excel Faylini Tanlash",  # "Browse Excel File" in Uzbek
            command=self.browse_file,
            width=180,
            height=40
        )
        self.browse_button.grid(row=0, column=1, padx=20, pady=10)
        
        # Progress section
        self.progress_frame = ctk.CTkFrame(self.content)
        self.progress_frame.grid(row=1, column=0, sticky="ew", padx=20, pady=10)
        self.progress_frame.grid_columnconfigure(0, weight=1)
        
        self.progress_label = ctk.CTkLabel(
            self.progress_frame, 
            text="Progress: 0%",
            font=ctk.CTkFont(size=18, weight="bold")
        )
        self.progress_label.grid(row=0, column=0, sticky="w", padx=20, pady=5)
        
        # Full-width progress bar
        self.progress_bar = ctk.CTkProgressBar(self.progress_frame, height=30)
        self.progress_bar.grid(row=1, column=0, sticky="ew", padx=20, pady=5)
        self.progress_bar.set(0)
        
        # Control buttons - spread across width
        self.button_frame = ctk.CTkFrame(self.content)
        self.button_frame.grid(row=2, column=0, sticky="ew", padx=20, pady=10)
        self.button_frame.grid_columnconfigure(1, weight=1)
        
        self.start_button = ctk.CTkButton(
            self.button_frame, 
            text="Boshlash",  # "Start" in Uzbek
            command=self.start_automation,
            state="disabled",
            width=180,
            height=40
        )
        self.start_button.grid(row=0, column=0, padx=10, pady=10)
        
        self.stop_button = ctk.CTkButton(
            self.button_frame, 
            text="To'xtatish",  # "Stop" in Uzbek
            command=self.stop_automation,
            state="disabled",
            width=180,
            height=40
        )
        self.stop_button.grid(row=0, column=1, padx=10, pady=10)
        
        self.reset_button = ctk.CTkButton(
            self.button_frame, 
            text="Qayta Sozlash",  # "Reset" in Uzbek
            command=self.reset_automation,
            width=180,
            height=40
        )
        self.reset_button.grid(row=0, column=2, padx=10, pady=10)
        
        # Log section
        self.log_frame = ctk.CTkFrame(self.content)
        self.log_frame.grid(row=3, column=0, sticky="nsew", padx=20, pady=(10, 20))
        self.log_frame.grid_columnconfigure(0, weight=1)
        self.log_frame.grid_rowconfigure(1, weight=1)
        
        self.log_label = ctk.CTkLabel(
            self.log_frame, 
            text="Faoliyat Logi",  # "Activity Log" in Uzbek
            font=ctk.CTkFont(size=16, weight="bold")
        )
        self.log_label.grid(row=0, column=0, sticky="w", padx=20, pady=10)
        
        self.log_text = ctk.CTkTextbox(
            self.log_frame, 
            height=500,
            font=ctk.CTkFont(size=14),
            wrap="word"
        )
        self.log_text.grid(row=1, column=0, sticky="nsew", padx=20, pady=(0, 20))

    def toggle_sidebar(self):
        if self.sidebar_open:
            self.sidebar.grid_forget()
            self.sidebar_open = False
        else:
            self.sidebar.grid(row=0, column=0, sticky="nsew", padx=0)
            self.sidebar_open = True

    def toggle_theme(self):
        if ctk.get_appearance_mode() == "Dark":
            ctk.set_appearance_mode("light")
            self.theme_switch.deselect()
        else:
            ctk.set_appearance_mode("dark")
            self.theme_switch.select()

    def show_dashboard(self):
        self.log_message("Boshqaruv paneliga o'tildi")  # "Switched to Dashboard view" in Uzbek

    def browse_file(self):
        self.file_path = filedialog.askopenfilename(
            filetypes=[("Excel Fayllari", "*.xlsx")]  # "Excel Files" in Uzbek
        )
        if self.file_path:
            self.file_label.configure(
                text=f"Tanlangan: {os.path.basename(self.file_path)}"  # "Selected" in Uzbek
            )
            self.start_button.configure(state="normal")
            self.log_message(f"Tanlangan fayl: {os.path.basename(self.file_path)}")  # "Selected file" in Uzbek

    def start_automation(self):
        if not self.is_running and self.file_path:
            self.is_running = True
            self.start_button.configure(state="disabled")
            self.stop_button.configure(state="normal")
            self.browse_button.configure(state="disabled")
            self.log_message("Login avtomatlashtirish boshlanyapti...")  # "Starting login automation..." in Uzbek
            
            # Run automation in a separate thread to keep the UI responsive
            automation_thread = threading.Thread(target=self.automate_login_from_excel)
            automation_thread.start()

    def stop_automation(self):
        self.is_running = False
        self.stop_button.configure(state="disabled")
        self.start_button.configure(state="normal")
        self.browse_button.configure(state="normal")
        self.log_message("Avtomatlashtirish to'xtatildi.")  # "Automation stopped" in Uzbek

    def reset_automation(self):
        self.is_running = False
        self.current_progress = 0
        self.progress_bar.set(0)
        self.progress_label.configure(text="Progress: 0%")
        self.start_button.configure(state="disabled")
        self.stop_button.configure(state="disabled")
        self.browse_button.configure(state="normal")
        self.file_label.configure(text="Fayl tanlanmagan")  # "No file selected" in Uzbek
        self.log_message("Avtomatlashtirish qayta tiklandi.")  # "Automation reset" in Uzbek

    def log_message(self, message):
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_entry = f"[{timestamp}] {message}"
        self.log_text.insert(tk.END, log_entry + "\n")
        self.log_text.yview(tk.END)
        print(log_entry)

    def automate_login_from_excel(self):
        try:
            df = pd.read_excel(self.file_path)
            user_credentials = [
                {"login": row["Login"], "password": row["Password"]}
                for _, row in df.iterrows()
            ]
            
            # Set Chrome options to open in a larger window
            chrome_options = Options()
            chrome_options.add_argument("--window-size=1200,800")  # Set window size to 1200x800
            chrome_options.add_argument("--start-maximized")  # Optional: Start maximized

            # Initialize WebDriver
            service = Service(CHROME_DRIVER_PATH)
            driver = webdriver.Chrome(service=service, options=chrome_options)
            wait = WebDriverWait(driver, 10)

            total_users = len(user_credentials)

            for i, user in enumerate(user_credentials, 1):
                if not self.is_running:
                    break
                try:
                    # Perform login actions here...
                    self.log_message(f"✅ {user['login']} - Muvaffaqiyatli kirib-chiqildi!")
                    driver.get(URL_EMAKTAB_LOGIN)
                    
                    login_field = wait.until(
                        EC.presence_of_element_located((By.NAME, "login"))
                    )
                    login_field.clear()
                    login_field.send_keys(user["login"])
                    
                    password_field = wait.until(
                        EC.presence_of_element_located((By.NAME, "password"))
                    )
                    password_field.clear()
                    password_field.send_keys(user["password"])
                    
                    password_field.send_keys(Keys.ENTER)
                    sleep(random.uniform(0.11, 1.12))
                    
                    logout_button = wait.until(
                        EC.element_to_be_clickable((By.LINK_TEXT, "Chiqish"))  # "Logout" in Uzbek
                    )
                    logout_button.click()
                    sleep(random.uniform(0.5, 1))
                    
                except Exception as e:
                    self.log_message(f"❌ {user['login']} - Xato: {str(e)}")  # "Error" in Uzbek
                finally:
                    progress_value = int((i / total_users) * 100)
                    self.update_progress(progress_value)

            driver.quit()  # Close the browser after all logins are processed
        except Exception as e:
            self.log_message(f"❌ Xato: {str(e)}")  # "Error" in Uzbek

    def update_progress(self, progress):
        self.current_progress = progress
        self.progress_bar.set(progress / 100)
        self.progress_label.configure(text=f"Progress: {progress}%")

if __name__ == "__main__":
    app = ModernLoginAutomation()
    app.app.mainloop()

[2024-12-25 12:34:47] Tanlangan fayl: emaktab.xlsx
[2024-12-25 12:34:47] Login avtomatlashtirish boshlanyapti...
[2024-12-25 12:34:48] ✅ doublekkyearner.xx77 - Muvaffaqiyatli kirib-chiqildi!
[2024-12-25 12:34:53] ✅ kaeyahh - Muvaffaqiyatli kirib-chiqildi!
